# Decorators
https://wiki.python.org/moin/PythonDecoratorLibrary

https://realpython.com/primer-on-python-decorators/#further-reading

## Simple example of what a decorator does

### Functions are first class objects - what does this mean?

In [10]:
# functions in python are first class objects, and so can be passed as arguments and returned from functions
def log_output():
    print("Logging something")
    

# this is assigning a function to a variable, not executing the function and storing the result
log = log_output

# this executest the function
log()

Logging something


In [11]:
# we can pass the function as a parameter to another function

def log_something(func):
    print("Before...")
    func() # invoke the function passed in as an argument
    print("After...")

# log is a function here
log_something(log)  

Before...
Logging something
After...


### Functions can be defined within functions, and returned - these are known as closures

In [ ]:
# we can now return a function from a function, this is called a closure

def make_case(case_type):
    
    def upper(value):
        return value.upper()

    def lower(value):
        return value.lower()
    

    if case_type == 'u':
        return upper
    else:
        return lower

In [33]:
# If we pass 'u' as the case_type parameter, the upper() closure is returned, otherwise the lower() closure is returned
func = make_case('u')
print(func('Hello'))

func = make_case('l')
print(func('Hello'))

HELLO
hello


In [34]:
# Now if we take this one step further and pass in a function, and return a closure which executes the function
# We can change the way the function passed in as an argument operates

def make_upper(func):
    
    def wrapper():
        # execute the function passed in as an argument
        result = func()
        
        # convert to upper case and return the result
        modified_result =  result.upper()
        return modified_result
    
    # return the modified function
    return wrapper


# this is a base function we want to modify
def greet():
    return "Hello there, how are you?"


greet()

# here we create modify the behaviour of greet() to force to upper case
greet = make_upper(greet)

# eecuting the function now, is exeuting the wrapper function around the original greet function
greet()


'HELLO THERE, HOW ARE YOU?'

## Decorator Syntax

In [35]:
# now we understand how a function can effetively return a function (a closure()), and we can pass in functions
# ...and therefore wrap behaviour around a function to alter its behaviour...we can introduce the decorator syntax

# the syntax demonstrated above, can be simplified using the @ decorator syntax
# this example is effectively exactly the same as: say_goodnight =  make_upper(say_goodnight)
@make_upper
def say_goodnight():
    return "Goodnight everyone"

say_goodnight()

'GOODNIGHT EVERYONE'

In [37]:
# we can now add @make_upper to any function which returns a string value, to alter its behaviour 
@make_upper
def say_something():
    return "Something!!"

say_something()

'SOMETHING!!'

### Applying multiple decorators

In [40]:
# multiple decorators can be applied to a function

# reverse a string
def reverse(func):
    
    def wrapper():
        result = func()
        result = result[::-1]
        return result
    
    return wrapper

# remove first character of a string
def remove_first(func):
    def wrapper():
        result = func()
        result = result[1:]
        return result
    
    return wrapper


# here we add multiple decorators, note how they are executed "upwards"
# i.e. the first character is removed and then the string is reversed
# this is the same as the syntax : process_string = reverse(remove_first(process_string))
@reverse
@remove_first
def process_string():
    return "Hello there!"

process_string()

'!ereht olle'

In [41]:
# here we add multiple decorators, note how they are executed "upwards"
# i.e. the string is reversed and THEN the first character is removed
# this is the same as the syntax : process_string = remove_first(reverse(process_string))
@remove_first
@reverse
def process_string():
    return "Hello there!"

process_string()

'ereht olleH'

In [43]:
# And of course we apply one, both or neither of the decorators
@remove_first
def process_string():
    return "Hello there!"

process_string()

'ello there!'

## Decorating a function with arguments

In [45]:
# the examples so far have not had to deal with decorating functions with arguments

def reverse(func):
    Decorators
    def wrapper():
        result = func()
        result = result[::-1]
        return result
    
    return wrapper


@reverse
def process_string(string_to_process):
    return string_to_process.upper();

In [47]:
# this wont work
try:
    process_string('Hello')
except Exception as e:
    print("Error: ", e)

Error:  wrapper() takes 0 positional arguments but 1 was given


In [51]:
# we solve this with *args and **kwargs
def reverse(func):
    
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        result = result[::-1]
        return result
    
    return wrapper

@reverse
def process_string(string_to_process):
    return string_to_process.upper();

print(process_string('Hello'))
print(process_string(string_to_process = 'Hello'))

OLLEH
OLLEH


## Trace example    

In [64]:
# this example allows us to trace each function call by printing out the function name being called, 
# any arguments and the return value
def trace(func):

    def wrapper(*args, **kwargs):
        print(f"TRACE: calling {func.__name__}() " 
              f"with args {args}, {kwargs}")
        original_result = func(*args, **kwargs)
        print(f"TRACE: {func.__name__}() " 
              f"returned {original_result!r}")
        return original_result
    
    return wrapper


@trace
def program_to_trace(value):
    return value[::-1].upper()

# all the arguments are positional and not keyword arguments
program_to_trace("I like you")
program_to_trace("What is your name?")

@trace
def another_program_to_trace(value):
    return value[::-1].lower()

# note how the trace picks up the keyword arguments this time
another_program_to_trace(value="I like you")
another_program_to_trace(value="What is your name?")

TRACE: calling program_to_trace() with args ('I like you',), {}
TRACE: program_to_trace() returned 'UOY EKIL I'
TRACE: calling program_to_trace() with args ('What is your name?',), {}
TRACE: program_to_trace() returned '?EMAN RUOY SI TAHW'
TRACE: calling another_program_to_trace() with args (), {'value': 'I like you'}
TRACE: another_program_to_trace() returned 'uoy ekil i'
TRACE: calling another_program_to_trace() with args (), {'value': 'What is your name?'}
TRACE: another_program_to_trace() returned '?eman ruoy si tahw'


'?eman ruoy si tahw'

## Preserving function metadata when decorating

In [10]:
# When we ue a decorator, we lost the metadata of the original funtion such as the function name and the docstring
from functools import wraps

def reverse(func):
   
    def wrapper(*args, **kwargs):
        
        print(func.__name__, func.__doc__)
        
        result = func(*args, **kwargs)
        result = result[::-1]
        return result
    
    return wrapper

@reverse
def process_string(string_to_process):
    '''
    Process a string in a particular way
    '''
    return string_to_process.upper();

process_string("Apple")

# here we can see that the name of the function is now 'wrapper' and the docstring is empty
print(process_string.__name__)
print(process_string.__doc__)

process_string 
    Process a string in a particular way
    
wrapper
None


In [11]:
# We can retain the metadata of the function being decorated using functools.wraps
from functools import wraps

def reverse(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        
        print(func.__name__, func.__doc__)
        
        result = func(*args, **kwargs)
        result = result[::-1]
        return result
    
    return wrapper

@reverse
def process_string(string_to_process):
    '''
    Process a string in a particular way
    '''
    return string_to_process.upper();

process_string("Apple")

# here we can see that the name of the function and docstring is retained
print(process_string.__name__)
print(process_string.__doc__)

process_string 
    Process a string in a particular way
    
process_string

    Process a string in a particular way
    


### Unwrapping a decorator

In [17]:
# Once we decorate a function, as long as we have used @wraps from functools, we can unwrap
# by accessing the __wrapped__ property

from functools import wraps

def reverse(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
       
        result = func(*args, **kwargs)
        result = result[::-1]
        return result
    
    return wrapper

@reverse
def process_string(string_to_process):
    '''
    Process a string in a particular way
    '''
    return string_to_process.upper();

print(process_string("Apple"))

# here was can acess the unwrapped function if we need to
unwrapped = process_string.__wrapped__
print(unwrapped("Apple"))

ELPPA
APPLE


## Decorator arguments @func(x,y)

In [9]:
# Passing arguments to a decorator, to change its behaviour can be achieved as follows

from functools import wraps
import logging

def logged(level, name=None, message=None):
    '''
    Add logging to a function.  
    - level is the logging level.
    - name is the logger
    - message is the log message
    If name and mesage are not provided, the module and function name are used by default.
    '''
    
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        log.setLevel(level)
        logmsg = message if message else func.__name__
        ch = logging.StreamHandler()
               
        # create formatter
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

        # add formatter to ch
        ch.setFormatter(formatter)

        # add ch to logger
        log.addHandler(ch)
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level,logmsg)
            return(func(*args, **kwargs))
        
        return wrapper
    return decorate


In [10]:
@logged(logging.DEBUG)
def add(x,y):
    return x + y

add(1,2)    

2021-07-15 08:13:24,431 - __main__ - DEBUG - add
2021-07-15 08:13:24,431 - __main__ - DEBUG - add
2021-07-15 08:13:24,431 - __main__ - DEBUG - add
2021-07-15 08:13:24,431 - __main__ - DEBUG - add


3

In [11]:
@logged(logging.CRITICAL,'example')
def spam():
    print('SPAM....!')

spam()

2021-07-15 08:13:25,791 - example - CRITICAL - spam
2021-07-15 08:13:25,791 - example - CRITICAL - spam
2021-07-15 08:13:25,791 - example - CRITICAL - spam


SPAM....!


## Slowing down code with a decorator

In [12]:
# this is an example of using a decorator to slow down a function using time.sleep()
# useful if polling a website for changes for example
import functools
import time

def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [14]:
countdown(10)

10
9
8
7
6
5
4
3
2
1
Liftoff!


In [19]:
# this is an example of using a decorator to slow down a function using time.sleep()
# useful if polling a website for changes for example
# this example passes a parameter to the decorator

import functools
import time

def slow_down(period):
    """Sleep 1 second before calling the function"""
   
    def decorator(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(period)  # this is where we use the decorator parameter
            return func(*args, **kwargs)
        return wrapper_slow_down
    
    return decorator
    

@slow_down(2)
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [20]:
countdown(5)  # run function every 5 seconds

5
4
3
2
1
Liftoff!


In [40]:
# this is an example of using a decorator to slow down a function using time.sleep()
# useful if polling a website for changes for example
# this example passes a parameter to the decorator, but the parameter is OPTIONAL

import functools
import time

def slow_down(_func=None, *, period=1):
    """Sleep given amount of seconds before calling the function"""
    
    """
        Here, the _func argument acts as a marker, noting whether the decorator has been called with arguments or not:
        If slow_down has been called without arguments, the decorated function will be passed in as _func. 
        If it has been called with arguments, then _func will be None, and some of the keyword arguments 
        may have been changed from their default values. 
        The * in the argument list means that the remaining arguments can’t be called as positional arguments.       
    """
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(period)
            return func(*args, **kwargs)
        return wrapper_slow_down

    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)
    
    

@slow_down(period=5)
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)
        
@slow_down
def countdown_default(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)
        


In [41]:
# In this case, the decorator was called with arguments. 
# Return a decorator function that can read and return a function.
countdown(10)

10
9
8
7
6
5
4
3
2
1
Liftoff!


In [39]:
# In this case, the decorator was called without decorator arguments. 
# Apply the decorator to the function immediately, which uses the default value for period of 1 second
countdown_default(10)

10
9
8
7
6
5
4
3
2
1
Liftoff!


## Stateful decorators

### Using functional attributes to collect state

In [43]:
# we can use the num_calls function attribute to track the number of times a decorated function is called
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_whee():
    print("Whee!")

In [44]:
say_whee()
say_whee()
say_whee()
say_whee()
say_whee()


Call 1 of 'say_whee'
Whee!
Call 2 of 'say_whee'
Whee!
Call 3 of 'say_whee'
Whee!
Call 4 of 'say_whee'
Whee!
Call 5 of 'say_whee'
Whee!


### Using classes to collect state

In [46]:
# classes can be used as decorators but they must be callable
# to make a class callable, the class must implement the __call__ method

class Counter:
    def __init__(self, start=0):
        self.count = start

    def __call__(self):
        self.count += 1
        print(f"Current count is {self.count}")

In [50]:
# Counter instances are now callable
counter1 = Counter()
counter1()
counter1()

counter2 = Counter()
counter2()
counter2()

Current count is 1
Current count is 2
Current count is 1
Current count is 2


In [53]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_whee():
    print("Whee!")
    
    

@CountCalls
def say_hello():
    print("Hello!")

In [54]:
say_whee()
say_whee()
say_whee()

Call 1 of 'say_whee'
Whee!
Call 2 of 'say_whee'
Whee!
Call 3 of 'say_whee'
Whee!


In [55]:
say_hello()
say_hello()
say_hello()

Call 1 of 'say_hello'
Hello!
Call 2 of 'say_hello'
Hello!
Call 3 of 'say_hello'
Hello!


In [56]:
say_whee()

Call 4 of 'say_whee'
Whee!


In [58]:
say_hello()
say_hello()
say_hello()
say_hello()
say_hello()

Call 5 of 'say_hello'
Hello!
Call 6 of 'say_hello'
Hello!
Call 7 of 'say_hello'
Hello!
Call 8 of 'say_hello'
Hello!
Call 9 of 'say_hello'
Hello!


In [59]:
say_whee()

Call 5 of 'say_whee'
Whee!


## Singleton decorator

In [62]:
# here is a singleton decorator which stores the first created instance of a class in a funtion attribute 'instance'
# subsequent requests to create an instance of the class, simply return the existing instance
import functools

def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton

@singleton
class TheOne:
    pass

In [63]:
# even though we create two instances, we can see they are the exact same instance 

c = TheOne()
d = TheOne()

c is d

True